In [1]:
import pandas as pd

df = pd.read_csv('/content/Ham-geog-updated1.csv')
df.head()

,,,Ham-geog-updated
Place Type,Location,"Lat, Long",Weight
City,London,"51.50673486004276, -0.12083646321188168",149
Village,Althrope,"52.28030002238975, -1.0015690158710147",4
City,Edinburgh,"55.948889, -3.190556",104
Castle,Bonhard,"56.41628189811223, -3.3812778745459653",1


In [3]:
import pandas as pd
df = pd.read_csv('/content/Ham-geog-updated1.csv', header=1)

if 'Ham-geog-updated' in df.columns:
    df = df.drop(columns=['Ham-geog-updated'])

df[['Latitude', 'Longitude']] = df['Lat, Long'].str.split(', ', expand=True)
df['Latitude'] = pd.to_numeric(df['Latitude'], errors='coerce')
df['Longitude'] = pd.to_numeric(df['Longitude'], errors='coerce')
df['Weight'] = pd.to_numeric(df['Weight'], errors='coerce')
df.dropna(subset=['Latitude', 'Longitude', 'Weight'], inplace=True)

print("DataFrame after cleaning and type conversion:")
df.info()
print("\nFirst 5 rows of the cleaned DataFrame:")
df.head()

DataFrame after cleaning and type conversion:
<class 'pandas.core.frame.DataFrame'>
Index: 131 entries, 0 to 136
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Place Type  131 non-null    object 
 1   Location    131 non-null    object 
 2   Lat, Long   131 non-null    object 
 3   Weight      131 non-null    float64
 4   Latitude    131 non-null    float64
 5   Longitude   131 non-null    float64
dtypes: float64(3), object(3)
memory usage: 7.2+ KB

First 5 rows of the cleaned DataFrame:


,Place Type,Location,"Lat, Long",Weight,Latitude,Longitude
0,City,London,"51.50673486004276, -0.12083646321188168",149.0,51.506735,-0.120836
1,Village,Althrope,"52.28030002238975, -1.0015690158710147",4.0,52.280300,-1.001569
2,City,Edinburgh,"55.948889, -3.190556",104.0,55.948889,-3.190556
3,Castle,Bonhard,"56.41628189811223, -3.3812778745459653",1.0,56.416282,-3.381278
4,Castle,Dublin Castle,"53.343007774364885, -6.267396216316814",4.0,53.343008,-6.267396


In [7]:
import folium

mean_latitude = df['Latitude'].mean()
mean_longitude = df['Longitude'].mean()

map = folium.Map(location=[mean_latitude, mean_longitude], zoom_start=6)

max_weight = df['Weight'].max()

def get_radius(weight):
    if max_weight == 0:
        return 5
    min_radius = 5
    max_radius = 30
    return min_radius + (max_radius - min_radius) * (weight / max_weight)

for idx, row in df.iterrows():
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=get_radius(row['Weight']),
        popup=f"<b>Location:</b> {row['Location']}<br><b>Place Type:</b> {row['Place Type']}<br><b>Weight:</b> {row['Weight']:.0f}",
        color='blue',
        fill=True,
        fill_color='blue'
    ).add_to(map)

min_lat = df['Latitude'].min()
max_lat = df['Latitude'].max()
min_lon = df['Longitude'].min()
max_lon = df['Longitude'].max()

map.fit_bounds([[min_lat, min_lon], [max_lat, max_lon]])

# Display the map
map